### Install dependencies

In [1]:
!pip install boto3 sagemaker langchain langchain-community langchain-core faiss-cpu requests opensearch-py sentence-transformers langchain-text-splitters requests-aws4auth qdrant-client -U


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install --upgrade --quiet  lark qdrant-client


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


### Load CSV data from S3

In [3]:
!pwd

/Users/robertgreer/Documents/Education/Berkeley/Classes/Datasci210_Capstone/Enterprise-RAG/notebooks


In [4]:
import boto3
import pandas as pd

s3 = boto3.client('s3')
bucket_name = 'recipes-rag'

In [5]:
file_key = 'recipes_w_cleaning_time_combined_features.parquet'
s3.download_file(bucket_name, file_key, f'../data/{file_key}')
df = pd.read_parquet(f'../data/{file_key}')

df.head()

,Name,RecipeCategory,Description,Keywords_string,RecipeIngredientQuantities,RecipeIngredientParts,RecipeInstructions,AggregatedRating,ReviewCount,CookTime_Minutes,PrepTime_Minutes,TotalTime_Minutes,Combined_Features,Combined_Features_Clean
0,Low-Fat Berry Blue Frozen Dessert,Frozen Desserts,Make and share this Low-Fat Berry Blue Frozen ...,Dessert Low Protein Low Cholesterol Healthy Fr...,"[4, 1⁄4, 1, 1]","[blueberries, granulated sugar, vanilla yogurt...",Toss 2 cups berries with sugar. Let stand for ...,4.5,4.0,1440,45,1485,Low-Fat Berry Blue Frozen Dessert Frozen Desse...,Low-Fat Berry Blue Frozen Dessert Frozen Desse...
1,Biryani,Chicken Breast,Make and share this Biryani recipe from Food.com.,Chicken Thigh & Leg Chicken Poultry Meat Asian...,"[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...","[saffron, milk, hot green chili peppers, onion...",Soak saffron in warm milk for 5 minutes and pu...,3.0,1.0,25,240,265,Biryani Chicken Breast Make and share this Bir...,Biryani Chicken Breast Make share Biryani reci...
2,Best Lemonade,Beverages,This is from one of my first Good House Keepi...,Low Protein Low Cholesterol Healthy Summer < 6...,"[1 1⁄2, 1, None, 1 1⁄2, None, 3⁄4]","[sugar, lemons, rind of, lemon, zest of, fresh...","Into a 1 quart Jar with tight fitting lid, put...",4.5,10.0,5,30,35,Best Lemonade Beverages This is from one of my...,Best Lemonade Beverages one first Good House K...
3,Carina's Tofu-Vegetable Kebabs,Soy/Tofu,This dish is best prepared a day in advance to...,Beans Vegetable Low Cholesterol Weeknight Broi...,"[12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1⁄2, 1⁄...","[extra firm tofu, eggplant, zucchini, mushroom...","Drain the tofu, carefully squeezing out excess...",4.5,2.0,20,1440,1460,Carina's Tofu-Vegetable Kebabs Soy/Tofu This d...,Carina's Tofu-Vegetable Kebabs Soy/Tofu dish b...
4,Cabbage Soup,Vegetable,Make and share this Cabbage Soup recipe from F...,Low Protein Vegan Low Cholesterol Healthy Wint...,"[46, 4, 1, 2, 1]","[plain tomato juice, cabbage, onion, carrots, ...",Mix everything together and bring to a boil. R...,4.5,11.0,30,20,50,Cabbage Soup Vegetable Make and share this Cab...,Cabbage Soup Vegetable Make share Cabbage Soup...


In [6]:
df.columns

Index(['Name', 'RecipeCategory', 'Description', 'Keywords_string',
       'RecipeIngredientQuantities', 'RecipeIngredientParts',
       'RecipeInstructions', 'AggregatedRating', 'ReviewCount',
       'CookTime_Minutes', 'PrepTime_Minutes', 'TotalTime_Minutes',
       'Combined_Features', 'Combined_Features_Clean'],
      dtype='object')

In [7]:
str(df.iloc[0]['Combined_Features_Clean'])

"Low-Fat Berry Blue Frozen Dessert Frozen Desserts Make share Low-Fat Berry Blue Frozen Dessert recipe Food.com. Toss 2 cups berries sugar. Let stand 45 minutes, stirring occasionally. Transfer berry-sugar mixture food processor. Add yogurt process smooth. Strain fine sieve. Pour baking pan (or transfer ice cream maker process according manufacturers' directions). Freeze uncovered edges solid centre soft. Transfer processor blend smooth again. Return pan freeze edges solid. Transfer processor blend smooth again. Fold remaining 2 cups blueberries. Pour plastic mold freeze overnight. Let soften slightly serve. Dessert Low Protein Low Cholesterol Healthy Free Of... Summer Weeknight Freezer Easy"

In [8]:
df = df[df['AggregatedRating'].notna()]
df.shape

(269294, 14)

### Load data into chunked documents

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

In [10]:
embedding_model = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")

/Users/robertgreer/Documents/Education/Berkeley/Classes/Datasci210_Capstone/Enterprise-RAG/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/robertgreer/Documents/Education/Berkeley/Classes/Datasci210_Capstone/Enterprise-RAG/venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


### Note: chunking may not be useful for us as the EDA has the following about description length
print(new_recipe['Combined_Features_Clean'].str.len().describe())


count    522517.000000\
mean        211.461774\
std         122.388646\
min          44.000000\
25%         131.000000\
50%         180.000000\
75%         255.000000\
max        4174.000000\
Name: Combined_Features_Clean, dtype: float64

In [ ]:
def create_textsplitter(chunks, overlaps):
    splits = {}
    for chunk in chunks:
        for overlap in overlaps:
            splits[f'chunk{str(chunk)}_overlap{str(overlap)}'] = RecursiveCharacterTextSplitter(chunk_size=chunk, chunk_overlap=overlap)
    return splits

In [ ]:
chunks = [128, 256, 512, 1024, 2048]
overlaps = [.25, .2, .15, .1]

text_splits = create_textsplitter(chunks, overlaps)

In [ ]:
text_splits['chunk1024_overlap0.25']

In [11]:
# def create_documents(df):
#     documents = []
#     for index, row in df.iterrows():
#         metadata = {
#             # 'recipe_id': str(row['RecipeId']) if not pd.isna(row['RecipeId']) else 'No ID Available',
#             'name': str(row['Name']) if not pd.isna(row['Name']) else 'No Name Available',
#             # 'cook_time': str(row['CookTime']) if not pd.isna(row['CookTime']) else 'No Cook Time Available',
#             # 'prep_time': str(row['PrepTime']) if not pd.isna(row['PrepTime']) else 'No Prep Time Available',
#             # 'total_time': str(row['TotalTime']) if not pd.isna(row['TotalTime']) else 'No Total Time Available',
#             'recipe_category': str(row['RecipeCategory']) if not pd.isna(row['RecipeCategory']) else 'No Category Available',
#             # 'keywords': str(row['Keywords']) if not pd.isna(row['Keywords']).all() else 'No Keywords Available',
#             'aggregated_rating': str(row['AggregatedRating']) if not pd.isna(row['AggregatedRating']) else 'No Rating Available',
#             'review_count': str(row['ReviewCount']) if not pd.isna(row['ReviewCount']) else 'No Reviews Available',
#             # 'calories': str(row['Calories']) if not pd.isna(row['Calories']) else 'No Calories Information Available',
#             # 'fat_content': str(row['FatContent']) if not pd.isna(row['FatContent']) else 'No Fat Content Available',
#             # 'saturated_fat_content': str(row['SaturatedFatContent']) if not pd.isna(row['SaturatedFatContent']) else 'No Saturated Fat Content Available',
#             # 'cholesterol_content': str(row['CholesterolContent']) if not pd.isna(row['CholesterolContent']) else 'No Cholesterol Content Available',
#             # 'sodium_content': str(row['SodiumContent']) if not pd.isna(row['SodiumContent']) else 'No Sodium Content Available',
#             # 'carbohydrate_content': str(row['CarbohydrateContent']) if not pd.isna(row['CarbohydrateContent']) else 'No Carbohydrate Content Available',
#             # 'sugar_content': str(row['SugarContent']) if not pd.isna(row['SugarContent']) else 'No Sugar Content Available',
#             # 'protein_content': str(row['ProteinContent']) if not pd.isna(row['ProteinContent']) else 'No Protein Content Available',
#             # 'recipe_servings': str(row['RecipeServings']) if not pd.isna(row['RecipeServings']) else 'No Servings Information Available',
#             # 'recipe_yield': str(row['RecipeYield']) if not pd.isna(row['RecipeYield']) else 'No Yield Information Available'
#         }

#         # Use Combined_Features_Clean for the document content
#         text = str(row['Combined_Features_Clean'])
#         doc = Document(page_content=text, metadata=metadata)
#         documents.append(doc)
        
#     return documents

def create_documents(df):
    df_copy = df.copy(deep=True)
    df = df.dropna(subset=["AggregatedRating"])
    df_copy = df_copy.fillna("")  # Convert NA values to empty strings
    df_copy = df_copy.astype(str)  # Cast all columns to string

    documents = []
    for _index, row in df_copy.iterrows():
        metadata = {
            "name": row["Name"] if row["Name"] else "No Name Available",
            "description": (
                row["Description"] if row["Description"] else "No Description Available"
            ),
            "recipe_category": (
                row["RecipeCategory"]
                if row["RecipeCategory"]
                else "No Category Available"
            ),
            "keywords": (
                row["Keywords_string"]
                if row["Keywords_string"]
                else "No Keywords Available"
            ),
            "recipe_ingredient_parts": (
                row["RecipeIngredientParts"]
                if row["RecipeIngredientParts"]
                else "No Recipe Ingredient Parts Available"
            ),
            "recipe_instructions": (
                row["RecipeInstructions"]
                if row["RecipeInstructions"]
                else "No Recipe Instructions Available"
            ),
            "aggregated_rating": (
                row["AggregatedRating"]
                if row["AggregatedRating"]
                else "No Rating Available"
            ),
            "review_count": (
                row["ReviewCount"] if row["ReviewCount"] else "No Reviews Available"
            ),
        }

        # List of fields to be included in the document content
        content_field = (
            row["Combined_Features"]
            if row["Combined_Features"]
            else "No Content Available"
        )

        # Create the document content using the combined features field
        doc = Document(page_content=content_field, metadata=metadata)
        documents.append(doc)

    return documents


In [12]:
# Take a sample of 100K receipes 
documents = create_documents(df.sample(n=40))

In [13]:
documents[0]

Document(metadata={'name': 'Chicken Livers With Mushrooms', 'description': "This should probably be called Chicken livers and gravy as it forms a wonderful tasting gravy.  I've had this recipe for years but don't remember where I got it. I'm posting the recipe as written and added my additions at the end.  Hope you like it, can be easily doubled or tripled.(though I don't double the margarine for cooking the livers)", 'recipe_category': 'Chicken Livers', 'keywords': 'Chicken Poultry Meat < 30 Mins Beginner Cook Easy Inexpensive', 'recipe_ingredient_parts': "['chicken liver' 'margarine' 'mushroom stems and pieces' 'water' 'flour'\n 'salt' 'pepper' 'parsley flakes' 'instant chicken bouillon'\n 'chicken bouillon cube']", 'recipe_instructions': "Boil water, add bouillon to dissolve, set aside. Melt margarine in 8 inch skillet. Brown livers over medium heat, stirring occasionally for 3-4 minutes. Stir in mushrooms (with liquid) Add bouillon/water, flour, salt(if using), pepper and parsley. 

In [14]:
len(documents)

40

In [ ]:
def split_documents_with_metadata(documents, text_splitter):
    split_docs = []
    for doc in documents:
        chunks = text_splitter.split_text(doc.page_content)
        for i, chunk in enumerate(chunks):
            split_docs.append(Document(page_content=chunk, metadata={**doc.metadata, "chunk_id": i}))
    return split_docs

In [ ]:
split_documents = {key: split_documents_with_metadata(documents, text_splitter_recursive) for key, text_splitter_recursive in text_splits.items()}

In [ ]:
split_documents['chunk256_overlap0.25'][0]

In [15]:
# We have > 500,000 recipes, this takes a long time to run
from langchain_community.vectorstores import Qdrant

# From documents with no chunking
qdrant_store = Qdrant.from_documents(documents,
    embedding_model,
    location=":memory:",
)

# from split documents (specify the chunk and overlap key)
# qdrant_store = Qdrant.from_documents(split_documents['chunk1024_overlap0.25'],
#     embedding_model,
#     location=":memory:",
# )

## Create llm query call

In [16]:
import json

In [17]:
bedrock_client = boto3.client('bedrock-runtime', region_name="us-east-1")

In [18]:
baseline_sys_prompt = """
You are a helpful assistant and expert in cooking recipes.

Before answering, always make at least one call to query_food_recipe_vector_db
to retrieve the relevant context of recipes and ingredients to generate an informed
and high-quality response to the user prompt but NEVER exceed a MAXIMUM of 
3 calls to the query_food_recipe_vector_db function.

Provide a response to the user prompt about food with recommended recipes and instructions.
"""

In [19]:
recipe_db_query_tool = {
  "name": "query_food_recipe_vector_db",
  "description": """
      Queries the vector database containing food recipes to retrieve the most relevant documents. 
      This function allows the model to generate and execute multiple queries as necessary to gather comprehensive context, 
      such as ingredients, preparation steps, and metadata like cuisine and diet type, ensuring accurate and thorough responses to user queries.
      """,
  "input_schema": {
    "type": "object",
    "properties": {
      "queries": {
        "type": "array",
        "items": {
          "type": "string",
          "description": "A query generated by the model to run against the vector database to fetch recipe documents."
        },
        "description": "A list of queries generated by the model to run against the vector database to fetch recipe documents."
      }
    },
    "required": ["queries"]
  }
}


In [20]:
MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"

def query_bedrock_llm(messages):
    response = bedrock_client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps({
            'anthropic_version': 'bedrock-2023-05-31', # This is required to use chat style messages object 
            'system': baseline_sys_prompt,
            'messages': messages,
            'max_tokens': 3000,
            "tools": [recipe_db_query_tool],

            # This config forces the model to always call the recipe db query tool atleast once 
            # https://docs.anthropic.com/en/docs/build-with-claude/tool-use#controlling-claudes-output
            # "tool_choice": {
            #     "type": "tool",
            #     "name": recipe_db_query_tool['name']
            # },
            
            # TODO: TUNE THESE VALUES
            'temperature': 0.1, 
            'top_p': 0.9
        })
    )
    response_body = json.loads(response.get('body').read())
    
    return response_body

## Create Retrieval Chainfrom langchain_core.prompts import ChatPromptTemplate

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableMap

In [22]:
def format_docs(docs):
    formatted_docs = []
    for doc in docs:
        formatted_docs.append(f"Metadata: {doc.metadata}\n")
    content = "\n\n".join(formatted_docs)
    
    return content

In [23]:
baseline_user_prompt = """
### Here is a user prompt:
{query}
"""

In [24]:
def process_prompt_basic(query_args):
    prompt_with_context = baseline_user_prompt.replace("{context}", query_args['context'])
    prompt_with_query = prompt_with_context.replace("{query}", query_args['query'])
    
    # This format doesn't matter much now, but we will use it later to 
    # persist chat history for continuous dialogue
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt_with_query
                }
            ]
        }
    ]
    
    return messages

## Create Initial Retriever

In [83]:

qdrant_retriever_rerank = qdrant_store.as_retriever(search_type='mmr', search_kwargs={"k": 5, 'lambda_mult': 0.5})


### Self Query

In [26]:
pip install langchain_openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [131]:
from langchain_openai import ChatOpenAI
import os

In [132]:
# os.environ["OPENAI_API_KEY"] = ""


In [133]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name="name",
        description="The name of the recipe",
        type="string",
    ),
    AttributeInfo(
        name="description",
        description="A brief description of the recipe",
        type="string",
    ),
    AttributeInfo(
        name="recipe_category",
        description="The category of the recipe, such as 'Quick Breads', 'Desserts', etc.",
        type="string",
    ),
    AttributeInfo(
        name="keywords",
        description="Keywords associated with the recipe",
        type="string",
    ),
    AttributeInfo(
        name="recipe_ingredient_parts",
        description="The ingredients required for the recipe",
        type="string",
    ),
    AttributeInfo(
        name="recipe_instructions",
        description="The instructions to prepare the recipe",
        type="string",
    ),
    AttributeInfo(
        name="aggregated_rating",
        description="The aggregated rating for the recipe",
        type="string",
    ),
    AttributeInfo(
        name="review_count",
        description="The number of reviews for the recipe",
        type="string",
    ),
]
document_content_description = "Detailed information about a recipe"

llm = ChatOpenAI(
    # model='gpt-4-turbo',
    model='gpt-4o',
    temperature=0,
    openai_api_key=os.environ["OPENAI_API_KEY"],
    
)

# qa_retriever = SelfQueryRetriever.from_llm(
#     llm, int_qdrant_store, document_content_description, metadata_field_info, verbose=True
# )


In [135]:
# output = qa_retriever.invoke("Recipes with no peanuts")
# 

### Partial Chain

In [136]:
def filtered_qdrant_store(documents=[]):
    fitered_qdrant_store = Qdrant.from_documents(documents,
        embedding_model,
        location=":memory:",
    )
    return fitered_qdrant_store

In [137]:
# message = 'Give me a chicken liver dish'
message = 'Give me a thai dish without peanuts'
coarse_search_docs = qdrant_retriever_rerank.invoke(message)
print(f'number of documents in coarse search: {len(coarse_search_docs)}')
int_qdrant_store = filtered_qdrant_store(coarse_search_docs)

qa_retriever = SelfQueryRetriever.from_llm(
    llm, filtered_qdrant_store(coarse_search_docs), document_content_description, metadata_field_info, verbose=True
)

number of documents in coarse search: 5


In [138]:
for doc in coarse_search_docs:
    print(doc.metadata['name'])

Fruit and Peanut Butter Wraps
Vermicelli and Paprika Soup (Dweda Zaara)
Roasted Red Pepper, Almond, and Garlic Dip
Banana Doghnuts
Easy Ginger Shrimp Wraps - HCG Phase 2


In [139]:
def qa_message_prompt(user_prompt):
    f"""
system: You are a helpful assistant and expert in cooking recipes.

You are evaluating for consistency of the user query.
Please retrieve documents that DO NOT violate dietary restrictions, allergies, 
or any requirements dictated by the user.

user: {user_prompt}
"""

In [140]:
results = qa_retriever.invoke(qa_message_prompt(message))

In [141]:
len(results)

4

In [142]:
for doc in results:
    print(doc.metadata['name'])

Vermicelli and Paprika Soup (Dweda Zaara)
Roasted Red Pepper, Almond, and Garlic Dip
Fruit and Peanut Butter Wraps
Banana Doghnuts


In [96]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
compressor = CrossEncoderReranker(model=reranker_model, top_n=1)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=qa_retriever
)

# compressed_docs = compression_retriever.invoke("What pizza can I make in 20 minutes?")
# compressed_docs

In [97]:
fine_search_docs = compression_retriever.invoke(message)

In [98]:
for doc in fine_search_docs:
    print(doc.metadata['name'])

Chicken Livers With Mushrooms


### Full chain

In [127]:
qdrant_rag_chain_rerank = (
    RunnableMap(
        {"context": qdrant_retriever_rerank | qa_retriever | format_docs ,
         "query": RunnablePassthrough()}
    )
    # | process_prompt_basic
    # | query_bedrock_llm
)

In [128]:
results = qdrant_rag_chain_rerank.invoke("Give me a peanut free Thai dish")

In [130]:
results['context']

'Metadata: {\'name\': \'Fruit and Peanut Butter Wraps\', \'description\': "A sweet lunchbox idea for fussy kids or serve as a healthy snack to improve their concentration. If you\'re looking for a savoury alternative, try spreading the tortilla with some houmous and add some strips of red pepper or grated carrot.", \'recipe_category\': \'Lunch/Snacks\', \'keywords\': \'Apple Tropical Fruits Fruit European Kid Friendly Sweet < 15 Mins Easy Inexpensive\', \'recipe_ingredient_parts\': "[\'banana\' \'lemon juice\' \'apple\' \'peanut butter\' \'whole wheat tortillas\']", \'recipe_instructions\': \'Peel and slice the banana very thinly. Place in a bowl and toss with the lemon juice. Quarter and core the apple. Slice into wafer thin slices and add to the bowl with the banana. Mix well. Warm the tortillas in the microwave for about 10 seconds. Spread 3tbsp of peanut butter over each tortilla. Place half the banana slices and half the apple slices over the top of each tortilla. Roll each one up